In [8]:
%load_ext autoreload
%autoreload 2
%aimport

!nvidia-smi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules to reload:
all-except-skipped

Modules to skip:

Thu Apr  1 16:22:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 440.59       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    35W / 300W |   4800MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+----------------------

- https://www.kaggle.com/boliu0/landmark-recognition-2020-third-place-submission
- https://github.com/haqishen/Google-Landmark-Recognition-2020-3rd-Place-Solution
- https://www.kaggle.com/zzy990106/b0-bert-cv0-9
- https://github.com/yang-zhang/product_category/blob/dev/notebooks/transformer_20210307E1--pin_memory.

In [10]:
HOME = "/data/git/shopee-product-matching"
p_out = f'{HOME}/output/dev0029'
!mkdir -p {p_out}
p_prp = f'{HOME}/output/prep001/train_prep.csv'

FOLD = 0
p_trained = f'{HOME}/output/dev0028'

import sys
sys.path.append(f"{HOME}/src")

import pandas as pd
from pl_model import ShpModel
import os
import torch
import numpy as np
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# inference model

In [29]:
import os
import cv2
import glob
import math
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import albumentations
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as tqdm

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

import geffnet

In [30]:
class ArcMarginProduct_subcenter(nn.Module):
    def __init__(self, in_features, out_features, k=3):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features*k, in_features))
        self.reset_parameters()
        self.k = k
        self.out_features = out_features
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        
    def forward(self, features):
        cosine_all = F.linear(F.normalize(features), F.normalize(self.weight))
        cosine_all = cosine_all.view(-1, self.out_features, self.k)
        cosine, _ = torch.max(cosine_all, dim=2)
        return cosine 

In [31]:
sigmoid = torch.nn.Sigmoid()
class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

class Swish_module(nn.Module):
    def forward(self, x):
        return Swish.apply(x)

    
 
    
class enet_arcface_FINAL(nn.Module):

    def __init__(self, enet_type, out_dim):
        super(enet_arcface_FINAL, self).__init__()
        self.enet = geffnet.create_model(enet_type.replace('-', '_'), pretrained=None)
        self.feat = nn.Linear(self.enet.classifier.in_features, 512)
        self.swish = Swish_module()
        self.metric_classify = ArcMarginProduct_subcenter(512, out_dim)
        self.enet.classifier = nn.Identity()
 
    def forward(self, x):
        x = self.enet(x)
        x = self.swish(self.feat(x))
        return F.normalize(x), self.metric_classify(x)
    
    
from rexnetv1 import ReXNetV1
from resnest.torch import resnest101    
class rex20_arcface(nn.Module):

    def __init__(self, enet_type, out_dim, load_pretrained=False):
        super(rex20_arcface, self).__init__()
        self.enet = ReXNetV1(width_mult=2.0)
        if load_pretrained:
            pretrain_wts = "/workspace/rexnetv1_2.0x.pth"            
            sd = torch.load(pretrain_wts)
            self.enet.load_state_dict(sd, strict=True)        
        
        self.feat = nn.Linear(self.enet.output[1].in_channels, 512)
        self.swish = Swish_module()
        self.metric_classify = ArcMarginProduct_subcenter(512, out_dim)
        self.enet.output = nn.Identity()
    
    def forward(self, x):
        x = self.enet(x)
        if x.ndim==1: 
            x = x.unsqueeze(0)          
        x = self.swish(self.feat(x))
        return F.normalize(x), self.metric_classify(x)    
    
class nest101_arcface(nn.Module):

    def __init__(self, enet_type, out_dim):
        super(nest101_arcface, self).__init__()
        self.enet = resnest101(pretrained=False)
        self.feat = nn.Linear(self.enet.fc.in_features, 512)
        self.swish = Swish_module()
        self.metric_classify = ArcMarginProduct_subcenter(512, out_dim)
        self.enet.fc = nn.Identity()    
    def forward(self, x):
        x = self.enet(x)
        x = self.swish(self.feat(x))
        return F.normalize(x), self.metric_classify(x)    

In [32]:
def load_model(model, model_file):
    state_dict = torch.load(model_file)
    if "model_state_dict" in state_dict.keys():
        state_dict = state_dict["model_state_dict"]
    state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
#     del state_dict['metric_classify.weight']
    model.load_state_dict(state_dict, strict=True)
    print(f"loaded {model_file}")
    model.eval()    
    return model

In [62]:
def load_model(model, plmodel):
    state_dict = plmodel.state_dict()
    if "model_state_dict" in state_dict.keys():
        state_dict = state_dict["model_state_dict"]
    state_dict = {k[6:] if k.startswith('model.') else k: state_dict[k] for k in state_dict.keys()}
#     del state_dict['metric_classify.weight']
    model.load_state_dict(state_dict, strict=True)
    model.eval()    
    return model

# load model

In [56]:
ls -hl {p_trained}/tensorboard_csv/*/checkpoints/*.ckpt

-rw-rw-r-- 1 ubuntu ubuntu 736M Mar 31 22:45 '/data/git/shopee-product-matching/output/dev0028/tensorboard_csv/0_0/checkpoints/epoch=22-step=9866.ckpt'
-rw-rw-r-- 1 ubuntu ubuntu 248M Apr  1 15:07 '/data/git/shopee-product-matching/output/dev0028/tensorboard_csv/1_1/checkpoints/epoch=28-step=12440.ckpt'


In [57]:
shpmodel = ShpModel.load_from_checkpoint(f'{p_trained}/tensorboard_csv/1_1/checkpoints/epoch=28-step=12440.ckpt')
out_dim = shpmodel.num_classes

In [58]:
device = torch.device('cuda')
batch_size = 4
num_workers = 4


In [50]:
foo = model_b0.state_dict()

In [51]:
foo.keys()

odict_keys(['enet.conv_stem.weight', 'enet.bn1.weight', 'enet.bn1.bias', 'enet.bn1.running_mean', 'enet.bn1.running_var', 'enet.bn1.num_batches_tracked', 'enet.blocks.0.0.conv_dw.weight', 'enet.blocks.0.0.bn1.weight', 'enet.blocks.0.0.bn1.bias', 'enet.blocks.0.0.bn1.running_mean', 'enet.blocks.0.0.bn1.running_var', 'enet.blocks.0.0.bn1.num_batches_tracked', 'enet.blocks.0.0.se.conv_reduce.weight', 'enet.blocks.0.0.se.conv_reduce.bias', 'enet.blocks.0.0.se.conv_expand.weight', 'enet.blocks.0.0.se.conv_expand.bias', 'enet.blocks.0.0.conv_pw.weight', 'enet.blocks.0.0.bn2.weight', 'enet.blocks.0.0.bn2.bias', 'enet.blocks.0.0.bn2.running_mean', 'enet.blocks.0.0.bn2.running_var', 'enet.blocks.0.0.bn2.num_batches_tracked', 'enet.blocks.1.0.conv_pw.weight', 'enet.blocks.1.0.bn1.weight', 'enet.blocks.1.0.bn1.bias', 'enet.blocks.1.0.bn1.running_mean', 'enet.blocks.1.0.bn1.running_var', 'enet.blocks.1.0.bn1.num_batches_tracked', 'enet.blocks.1.0.conv_dw.weight', 'enet.blocks.1.0.bn2.weight', 'ene

In [52]:
bar = shpmodel.state_dict()

In [53]:
foo.keys()

odict_keys(['enet.conv_stem.weight', 'enet.bn1.weight', 'enet.bn1.bias', 'enet.bn1.running_mean', 'enet.bn1.running_var', 'enet.bn1.num_batches_tracked', 'enet.blocks.0.0.conv_dw.weight', 'enet.blocks.0.0.bn1.weight', 'enet.blocks.0.0.bn1.bias', 'enet.blocks.0.0.bn1.running_mean', 'enet.blocks.0.0.bn1.running_var', 'enet.blocks.0.0.bn1.num_batches_tracked', 'enet.blocks.0.0.se.conv_reduce.weight', 'enet.blocks.0.0.se.conv_reduce.bias', 'enet.blocks.0.0.se.conv_expand.weight', 'enet.blocks.0.0.se.conv_expand.bias', 'enet.blocks.0.0.conv_pw.weight', 'enet.blocks.0.0.bn2.weight', 'enet.blocks.0.0.bn2.bias', 'enet.blocks.0.0.bn2.running_mean', 'enet.blocks.0.0.bn2.running_var', 'enet.blocks.0.0.bn2.num_batches_tracked', 'enet.blocks.1.0.conv_pw.weight', 'enet.blocks.1.0.bn1.weight', 'enet.blocks.1.0.bn1.bias', 'enet.blocks.1.0.bn1.running_mean', 'enet.blocks.1.0.bn1.running_var', 'enet.blocks.1.0.bn1.num_batches_tracked', 'enet.blocks.1.0.conv_dw.weight', 'enet.blocks.1.0.bn2.weight', 'ene

In [54]:
bar.keys()

odict_keys(['model.enet.conv_stem.weight', 'model.enet.bn1.weight', 'model.enet.bn1.bias', 'model.enet.bn1.running_mean', 'model.enet.bn1.running_var', 'model.enet.bn1.num_batches_tracked', 'model.enet.blocks.0.0.conv_dw.weight', 'model.enet.blocks.0.0.bn1.weight', 'model.enet.blocks.0.0.bn1.bias', 'model.enet.blocks.0.0.bn1.running_mean', 'model.enet.blocks.0.0.bn1.running_var', 'model.enet.blocks.0.0.bn1.num_batches_tracked', 'model.enet.blocks.0.0.se.conv_reduce.weight', 'model.enet.blocks.0.0.se.conv_reduce.bias', 'model.enet.blocks.0.0.se.conv_expand.weight', 'model.enet.blocks.0.0.se.conv_expand.bias', 'model.enet.blocks.0.0.conv_pw.weight', 'model.enet.blocks.0.0.bn2.weight', 'model.enet.blocks.0.0.bn2.bias', 'model.enet.blocks.0.0.bn2.running_mean', 'model.enet.blocks.0.0.bn2.running_var', 'model.enet.blocks.0.0.bn2.num_batches_tracked', 'model.enet.blocks.1.0.conv_pw.weight', 'model.enet.blocks.1.0.bn1.weight', 'model.enet.blocks.1.0.bn1.bias', 'model.enet.blocks.1.0.bn1.runni

In [63]:
model_b0 = enet_arcface_FINAL('tf_efficientnet_b0_ns', out_dim=out_dim).to(device)
model_b0 = load_model(model_b0, shpmodel)


# old

In [2]:
HOME = "/data/git/shopee-product-matching"
p_out = f'{HOME}/output/dev0028'
!mkdir -p {p_out}
p_prp = f'{HOME}/output/prep001/train_prep.csv'

FOLD = 0


import sys
sys.path.append(f"{HOME}/src")

import pandas as pd
from argparse import ArgumentParser
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pl_model import ShpModel, ShpDataModule
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
parser = ArgumentParser()
parser.add_argument(
    "--train_batch_size",
    help="How many samples per batch to load for train dataloader.",
    type=int,
    default=64,
)
parser.add_argument(
    "--val_batch_size",
    help="How many samples per batch to load for validation dataloader.",
    type=int,
    default=128,
)
parser.add_argument(
    "--dataloader_num_workers",
    help="How many subprocesses to use for data loading. 0 means that the data will be loaded in the main process.",
    type=int,
    default=8,
)
parser.add_argument(
    "--pin_memory",
    help="Wether to use pin_memory in pytorch dataloader. If True, the data loader will copy Tensors into CUDA pinned memory before returning them.",
    action="store_true",
)

parser = pl.Trainer.add_argparse_args(parser)
parser = ShpModel.add_model_specific_args(parser)
args_list = [
    '--default_root_dir', p_out,
    '--kernel-type', '',
    '--enet-type', 'tf_efficientnet_b0_ns',

]

args = parser.parse_args(args_list)

In [4]:
df = pd.read_csv(p_prp)
display(df.head(3))
dftrn = df[df.fold!=FOLD].copy()
dfval = df[df.fold==FOLD].copy()
dftrn.shape, dfval.shape

data_module = ShpDataModule(
    dftrn=dftrn,
    dfval=dfval,
    train_batch_size=args.train_batch_size,
    val_batch_size=args.val_batch_size,
    pin_memory=args.pin_memory,    
    
)

,posting_id,image,image_phash,title,label_group,fold,filepath
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,0,/data/git/shopee-product-matching/input/shopee...
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,2,/data/git/shopee-product-matching/input/shopee...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,0,/data/git/shopee-product-matching/input/shopee...


In [5]:
data_module.setup()
data_module.num_classes

11014

In [6]:
shpmodel = ShpModel(
    kernel_type=args.kernel_type,
    enet_type=args.enet_type,
    learning_rate=args.learning_rate,
    num_classes=data_module.num_classes,
    margins=data_module.margins,
)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b0_ns-c0e6a31c.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/tf_efficientnet_b0_ns-c0e6a31c.pth


In [7]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger

csv_logger = CSVLogger(p_out, name='csv')
tb_logger = TensorBoardLogger(p_out, name='tensorboard')

trainer = pl.Trainer.from_argparse_args(args, 
#                                         limit_train_batches=10, limit_val_batches=2, 
#                                         fast_dev_run=True,
                                        max_epochs=50,
                                        callbacks=[EarlyStopping(monitor='valid_loss')],
                                        stochastic_weight_avg=True,
                                        log_gpu_memory=True, 
                                        gpus=1,
                                        logger=[tb_logger,csv_logger],
                                       )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
pl.seed_everything(1234)
trainer.fit(shpmodel, data_module)

Global seed set to 1234
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                      | Params
-------------------------------------------------------
0 | model    | Effnet_Landmark           | 21.6 M
1 | arc      | ArcFaceLossAdaptiveMargin | 0     
2 | accuracy | Accuracy                  | 0     
-------------------------------------------------------
21.6 M    Trainable params
0         Non-trainable params
21.6 M    Total params
86.324    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/data/anaconda3/envs/shopee/lib/python3.7/site-packages/pytorch_lightning/core/step_result.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!find $p_out/

In [ ]:
!find $p_out/tensorboard/

In [ ]:
ls -hl {p_out}/tensorboard_csv/*/checkpoints/*.ckpt

In [ ]:
!nvidia-smi

# load model

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

from utils import meanf1, get_targets, get_preds_by_thrsh, preds2pids
from neighbor import get_nbrs

In [ ]:
device = torch.device('cuda')

shpmodel = ShpModel.load_from_checkpoint(f'{p_out}/tensorboard_csv/0_0/checkpoints/epoch=21-step=9437.ckpt')
shpmodel = shpmodel.eval()
shpmodel = shpmodel.to(device)

# dataloader

In [ ]:
from dataset import ShopeeDataset, get_transforms
from torch.utils.data import DataLoader

df = pd.read_csv(p_prp)

pids = df.posting_id.values

tfms_trn, tfms_val = get_transforms(224)
ds = ShopeeDataset(df, mode="test", transform=tfms_val)
dl = DataLoader(ds,
            batch_size=128,
            num_workers=8,
            pin_memory=True,
        )

# get feats

In [ ]:
for dat in dl:
    dat = dat.to(device)
    with torch.no_grad():
        feat = shpmodel(dat)
    break

feat.shape

In [ ]:
feats = np.zeros((len(ds), feat.shape[1]))
i = 0
for dat in tqdm(dl):
    with torch.no_grad():
        dat = dat.to(device)
        output = shpmodel(dat)
        l = len(output)
        feats[i : i + l, :] = output.cpu().detach().numpy()
        i += l

In [ ]:
feats.shape

# verify

In [ ]:
dists, idx = get_nbrs(feats)

In [ ]:
preds = get_preds_by_thrsh(dists, idx, thrsh=2.)
preds = preds2pids(preds, pids)

In [ ]:
targets = get_targets(df)

In [ ]:
meanf1(preds,targets)

In [ ]:
thrshes = np.linspace(1.75,2.25,num=11)
print(thrshes)

In [ ]:
f1_lst = []
for thrsh in tqdm(thrshes):
    preds = get_preds_by_thrsh(dists, idx, thrsh)
    preds = preds2pids(preds, pids)
    f1_lst.append(meanf1(preds,targets))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(thrshes, f1_lst)  

In [ ]:
def find_best(dists, idx, thrshes = np.linspace(1.75,2.25,num=11)):
    f1_lst = []
    preds_lst = []
    for thrsh in tqdm(thrshes):
        preds = get_preds_by_thrsh(dists, idx, thrsh)
        preds = preds2pids(preds, pids)
        preds_lst.append(preds)
        f1 = meanf1(preds,targets)
        f1_lst.append(f1)
    f1_best, thrsh_best, preds_best = sorted(zip(f1_lst, thrshes, preds_lst), reverse=True)[0]
    return f1_best, thrsh_best, preds_best

In [ ]:
f1_best, thrsh_best, preds_best = find_best(dists, idx, thrshes)

In [ ]:
f1_best, thrsh_best